In [ ]:
pip install pyserial sympy numpy

In [ ]:
import serial
import time
from sympy import *
import numpy as np

# 全局几何参数（根据参考程序推导）
d1 = 70   # 基座高度
a2 = 120  # 第二关节臂长
a3 = 120  # 第三关节臂长  
a4 = 80   # 第四关节臂长
d5 = 150  # 末端执行器长度

print("=== Braccio机器人逆向运动学实验 ===")
print("步骤1: 建立串行通信连接")

# 任务1: 串行控制机器人
def establish_serial_connection():
    """建立与机器人的串行连接"""
    try:
        # 修改COM端口号以匹配您的设备
        port = input("请输入COM端口号（例如：COM7）: ").strip()
        if not port:
            port = 'COM7'  # 默认端口
        
        s = serial.Serial(port, 115200, timeout=5)
        time.sleep(3)  # 等待连接稳定
        print(f"✓ 成功连接到 {port}")
        return s
    except Exception as e:
        print(f"✗ 连接失败: {e}")
        return None

def move_to_home_position(serial_conn):
    """移动到初始位置"""
    if serial_conn:
        # 发送初始位置命令: base=90, shoulder=90, elbow=0, wrist=90, twist=0
        command = b'P90,90,0,90,0,100,30\n'
        serial_conn.write(command)
        response = serial_conn.readline().decode().strip()
        print(f"机器人响应: {response}")
        return response == "OK"
    return False

print("\n步骤2: 角度转换函数")

# 角度转换函数
def deg2rad(degrees):
    """度数转弧度"""
    return degrees * pi / 180

def rad2deg(radians):
    """弧度转度数"""
    return radians * 180 / pi

print("\n步骤3: 逆向运动学核心函数")

def BraccioInverse(w):
    """
    Braccio机器人逆向运动学函数
    参数: w = [x, y, z] - 目标位置向量
    返回: q = [None, q1, q2, q3, q4, q5] - 关节角度向量（弧度）
    注意: 索引从1开始，符合算法要求
    """
    
    # 创建关节角度向量，索引从1开始
    q = [None] * 6  # q[0]不使用，q[1]到q[5]存储关节角度
    
    x, y, z = w[0], w[1], w[2]
    
    print(f"目标位置: x={x}, y={y}, z={z}")
    
    try:
        # 计算 q[1] (基座旋转角)
        q[1] = atan2(y, x)
        print(f"q[1] (基座) = {float(q[1])} rad = {float(rad2deg(q[1]))} deg")
        
        # 计算平面距离
        r = sqrt(x**2 + y**2)
        print(f"平面距离 r = {float(r)}")
        
        # 计算到末端的距离（考虑末端执行器长度）
        z_effective = z - d1  # 减去基座高度
        r_effective = r
        
        # 到腕关节的距离
        distance_to_wrist = sqrt(r_effective**2 + z_effective**2) - a4
        
        if distance_to_wrist <= 0:
            print("目标位置太近，无法到达")
            return None
            
        print(f"到腕关节的有效距离 = {float(distance_to_wrist)}")
        
        # 使用余弦定理计算 q[3] (肘关节角)
        cos_q3 = (distance_to_wrist**2 - a2**2 - a3**2) / (2 * a2 * a3)
        
        if abs(cos_q3) > 1:
            print(f"余弦值超出范围: {float(cos_q3)}")
            return None
            
        q[3] = acos(cos_q3)
        print(f"q[3] (肘关节) = {float(q[3])} rad = {float(rad2deg(q[3]))} deg")
        
        # 计算 q[2] (肩关节角)
        beta = atan2(z_effective, r_effective)
        alpha = atan2(a3 * sin(q[3]), a2 + a3 * cos(q[3]))
        q[2] = beta - alpha
        print(f"q[2] (肩关节) = {float(q[2])} rad = {float(rad2deg(q[2]))} deg")
        
        # 计算 q[4] (腕关节角) - 保持末端水平
        q[4] = -(q[2] + q[3])
        print(f"q[4] (腕关节) = {float(q[4])} rad = {float(rad2deg(q[4]))} deg")
        
        # q[5] (扭转角) - 通常设为0
        q[5] = 0
        print(f"q[5] (扭转) = {float(q[5])} rad = {float(rad2deg(q[5]))} deg")
        
        return q
        
    except Exception as e:
        print(f"逆向运动学计算错误: {e}")
        return None

print("\n步骤4: 角度调整和命令生成函数")

def adjust_angles_for_braccio(q):
    """
    根据Braccio机器人的实际旋转方向调整角度
    参考程序中的角度补偿
    """
    if q is None:
        return None
    
    # 调整后的角度（度数）
    adjusted = [None] * 6
    
    # q[1]: 基座 - 添加180度偏移（参考程序中的 +pi）
    adjusted[1] = float(rad2deg(q[1])) + 180
    
    # q[2]: 肩关节 - 直接转换
    adjusted[2] = float(rad2deg(q[2]))
    
    # q[3]: 肘关节 - 减去90度偏移（参考程序中的 -pi/2）
    adjusted[3] = float(rad2deg(q[3])) - 90
    
    # q[4]: 腕关节 - 减去90度偏移
    adjusted[4] = float(rad2deg(q[4])) - 90
    
    # q[5]: 扭转 - 减去90度偏移
    adjusted[5] = float(rad2deg(q[5])) - 90
    
    return adjusted

def genBraccioString(q):
    """
    生成发送给机器人控制器的命令字符串
    参数: q - 关节角度向量（弧度）
    返回: 命令字符串或None（如果位置不可达）
    """
    
    if q is None:
        return None
    
    # 调整角度
    adjusted_angles = adjust_angles_for_braccio(q)
    
    if adjusted_angles is None:
        return None
    
    # 检查角度范围（0-180度）
    for i in range(1, 6):
        angle = adjusted_angles[i]
        if angle < 0 or angle > 180:
            print(f"角度 q[{i}] = {angle}° 超出伺服电机范围 [0°, 180°]")
            return None
    
    # 生成命令字符串
    command = f"P{int(adjusted_angles[1])},{int(adjusted_angles[2])},{int(adjusted_angles[3])},{int(adjusted_angles[4])},{int(adjusted_angles[5])},130,50"
    
    print(f"生成的命令: {command}")
    return command

print("\n步骤5: 测试函数")

def test_inverse_kinematics():
    """测试逆向运动学实现"""
    
    print("\n=== 逆向运动学测试 ===")
    
    # 测试点1: 前方200mm，高度40mm（参考程序中的测试点）
    test_positions = [
        [200, 0, 40],      # 正前方
        [150, 150, 100],   # 右前方上方
        [100, -100, 80],   # 左前方
        [180, 50, 120],    # 右前方高处
    ]
    
    for i, pos in enumerate(test_positions, 1):
        print(f"\n--- 测试点 {i}: {pos} ---")
        
        # 计算逆向运动学
        q = BraccioInverse(pos)
        
        if q is not None:
            # 生成命令字符串
            command = genBraccioString(q)
            
            if command is not None:
                print(f"✓ 位置可达，命令: {command}")
            else:
                print("✗ 角度超出范围")
        else:
            print("✗ 位置不可达")

def interactive_control(serial_conn):
    """交互式控制模式"""
    
    print("\n=== 交互式控制模式 ===")
    print("输入目标位置坐标，程序将计算逆向运动学并控制机器人")
    print("输入格式: x,y,z （例如: 200,0,40）")
    print("输入 'quit' 退出")
    
    while True:
        try:
            user_input = input("\n请输入目标位置: ").strip()
            
            if user_input.lower() == 'quit':
                break
            
            # 解析输入
            coords = [float(x.strip()) for x in user_input.split(',')]
            
            if len(coords) != 3:
                print("请输入三个坐标值: x,y,z")
                continue
            
            print(f"目标位置: [{coords[0]}, {coords[1]}, {coords[2]}]")
            
            # 计算逆向运动学
            q = BraccioInverse(coords)
            
            if q is not None:
                command = genBraccioString(q)
                
                if command is not None:
                    print(f"计算成功！命令: {command}")
                    
                    # 发送命令到机器人
                    if serial_conn:
                        confirm = input("是否发送命令到机器人？(y/n): ").strip().lower()
                        if confirm == 'y':
                            serial_conn.write(f"{command}\n".encode('ascii'))
                            response = serial_conn.readline().decode().strip()
                            print(f"机器人响应: {response}")
                        else:
                            print("命令未发送")
                    else:
                        print("串行连接未建立，无法发送命令")
                else:
                    print("角度超出机器人工作范围")
            else:
                print("目标位置不可达")
                
        except ValueError:
            print("输入格式错误，请使用 x,y,z 格式")
        except Exception as e:
            print(f"错误: {e}")

print("\n步骤6: 主程序")

def main():
    """主程序"""
    
    print("开始Braccio机器人逆向运动学实验")
    
    # 首先进行测试（不需要实际机器人连接）
    test_inverse_kinematics()
    
    # 询问是否连接实际机器人
    connect_robot = input("\n是否连接实际机器人进行测试？(y/n): ").strip().lower()
    
    serial_conn = None
    if connect_robot == 'y':
        serial_conn = establish_serial_connection()
        
        if serial_conn:
            print("移动到初始位置...")
            if move_to_home_position(serial_conn):
                print("✓ 机器人已移动到初始位置")
            else:
                print("✗ 移动到初始位置失败")
    
    # 交互式控制
    interactive_control(serial_conn)
    
    # 关闭串行连接
    if serial_conn:
        serial_conn.close()
        print("串行连接已关闭")

# 运行主程序
if __name__ == "__main__":
    main()